In [1]:
import omega as o
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from tqdm.auto import tqdm

In [37]:
def variance(weights, prices):
    returns = np.log(prices).diff()[1:]
    cov = returns.cov() * 252
    return weights @ cov @ weights.T

def sharpe_ratio(weights, prices):
    returns = np.log(prices).diff()[1:]
    portfolio_returns = returns @ weights.T
    expected_return = np.mean(portfolio_returns) * 252
    portfolio_std = np.sqrt(variance(weights, prices))
    return (expected_return) / portfolio_std 

def neg_sharpe_ratio(weights, prices):
    return -sharpe_ratio(weights, prices)

def risk_contributions(weights, prices):
    returns = np.log(prices).diff()[1:]
    cov = returns.cov() * 252
    return (weights *(cov @ weights.T)) / variance(weights, prices)

def risk_parity_obj(weights, prices):
    num_assets = len(prices.columns)
    asset_contributions = risk_contributions(weights, prices)
    equal_contribution = 1.0 / num_assets
    risk_parity_obj = np.sum((asset_contributions - equal_contribution)**2)
    return risk_parity_obj

In [36]:
class OptimizePortfolio:

    def __init__(self, prices, shorts=False, method='max-sharpe'):
        self.stocks = prices.columns.to_list()
        self.prices = prices
        self.shorts = shorts
        self.bounds = [(-1, 1) if self.shorts else (0, 1) for _ in range(len(self.stocks))]
        self.constraints = ({'type': 'eq', 'fun': lambda w: np.sum(np.abs(w)) - 1})
        self.initial_weights = np.array([1/len(self.stocks)] * len(self.stocks))
        self.methods = {
            'max-sharpe': neg_sharpe_ratio, 
            'min-variance': variance, 
            'risk-parity': risk_parity_obj
        }
        self.objective = self.methods[method]

    def serialize(self):
        return pd.Series(self.weights, index=self.stocks)

    def optimize(self):

        opt = minimize(
            lambda w: self.objective(weights=w, prices=self.prices),
            self.initial_weights,
            bounds=self.bounds,
            constraints=self.constraints,
        )

        self.weights = opt.x
        self.sharpe = sharpe_ratio(self.weights, self.prices)
        self.risk = np.sqrt(variance(self.weights, self.prices))
        self.risk_contributions = risk_contributions(self.weights, self.prices)